In [2]:
import chalenge1000
import model_lib
import collections

def vocab(element):
    if type(element) == str:
        return ' '.join(set(element.split(' ')))
    else:
        return ''

col_count = lambda x: ' '.join([a for a,b in collections.Counter(x.split()).most_common(10000)])
lemm = lambda x: model_lib.Lemm(x,file = 'stopwords_agri.txt').X


Chalenge = chalenge1000.Native()

df = Chalenge.X
df['corpus'] = df.join(Chalenge.descriptions_trad)[['prez_struc','prez_produit_struc']].agg(sum, axis = 1)
df['type'] = df['corpus'].map(type)
df['corpus'] = df['corpus'].map(vocab)
df['corpus_lemm'] = df['corpus'].map(lemm)
df['collection_counter'] = df['corpus_lemm'].map(col_count)
df = df[df['type'] ==str]

In [3]:
agriculture = df[df['categorie'] == 'agriculture']
others = df[df['categorie'].isin([ 'media'])]

corpus_agri = agriculture['collection_counter'].agg(sum, axis = 0)
#corpus_agri = model_lib.Lemm(corpus_agri,file = 'stopwords_agri.txt').X
corpus_others = others['collection_counter'].agg(sum, axis = 0)
#corpus_others = model_lib.Lemm(corpus_others,file = 'stopwords_agri.txt').X

counter_agri = collections.Counter(corpus_agri.split()).most_common(5000)
counter_others = collections.Counter(corpus_others.split()).most_common(5000)

counter_agri = list({(a,b) for a,b in counter_agri}.union({(k,0) for k,l in counter_others if k not in [a for a,b in counter_agri]}))
counter_others = list({(a,b) for a,b in counter_others}.union({(k,0) for k,l in counter_agri if k not in [a for a,b in counter_others]}))
counter_agri = {(a, b*100/len(agriculture)) for a,b in counter_agri}
counter_others = {(a, b*100/len(others)) for a,b in counter_others}

import operator

id = operator.itemgetter(0)  # To get id field.

idinfo = {id(rec): rec[1:] for rec in counter_agri}  # Dict for fast look-ups.

merged = [info + idinfo[id(info)] for info in counter_others if id(info) in idinfo]

merged.sort(key=lambda x: x[2], reverse = True)

map_vocab = [(a,c,b, c-b) for a,b,c in merged]
map_vocab.sort(key=lambda x: x[-1], reverse = True)

import pandas

keyword_map = pandas.DataFrame(map_vocab, columns = ['keyword','agriculture', 'others', 'difference']).sort_values('difference')

keyword_map.describe(include= 'all')

keyword_map.sort_values('difference', ascending = False)


,keyword,agriculture,others,difference
0,agricultural,36.834532,2.173913,34.660619
1,food,37.122302,4.347826,32.774476
2,farmer,29.640288,1.086957,28.553331
3,farm,28.057554,0.000000,28.057554
4,agriculture,26.906475,2.173913,24.732562
...,...,...,...,...
6188,graphic,0.000000,17.391304,-17.391304
6189,service,25.035971,46.739130,-21.703159
6190,agency,2.589928,25.000000,-22.410072
6191,communication,4.316547,32.608696,-28.292149


In [4]:
keyword_map.sort_values('difference', ascending = False)

,keyword,agriculture,others,difference
0,agricultural,36.834532,2.173913,34.660619
1,food,37.122302,4.347826,32.774476
2,farmer,29.640288,1.086957,28.553331
3,farm,28.057554,0.000000,28.057554
4,agriculture,26.906475,2.173913,24.732562
...,...,...,...,...
6188,graphic,0.000000,17.391304,-17.391304
6189,service,25.035971,46.739130,-21.703159
6190,agency,2.589928,25.000000,-22.410072
6191,communication,4.316547,32.608696,-28.292149


In [5]:
subset_1 = keyword_map[(keyword_map['difference'] > 15) |  (keyword_map['difference'] < -15) ]#|  (keyword_map['agriculture'] > 15)]

subset_1.sort_values('difference', ascending = False)

,keyword,agriculture,others,difference
0,agricultural,36.834532,2.173913,34.660619
1,food,37.122302,4.347826,32.774476
2,farmer,29.640288,1.086957,28.553331
3,farm,28.057554,0.000000,28.057554
4,agriculture,26.906475,2.173913,24.732562
5,product,55.971223,31.521739,24.449484
6,produce,36.115108,14.130435,21.984673
7,production,41.294964,21.739130,19.555834
8,rural,21.582734,3.260870,18.321864
9,crop,17.985612,0.000000,17.985612


In [6]:
subset_1 = keyword_map[(keyword_map['difference'] > 15) |  (keyword_map['difference'] < -15) |  (keyword_map['agriculture'] > 15)]

subset_1.sort_values('difference', ascending = False)

,keyword,agriculture,others,difference
0,agricultural,36.834532,2.173913,34.660619
1,food,37.122302,4.347826,32.774476
2,farmer,29.640288,1.086957,28.553331
3,farm,28.057554,0.000000,28.057554
4,agriculture,26.906475,2.173913,24.732562
5,product,55.971223,31.521739,24.449484
6,produce,36.115108,14.130435,21.984673
7,production,41.294964,21.739130,19.555834
8,rural,21.582734,3.260870,18.321864
9,crop,17.985612,0.000000,17.985612


In [7]:
import numpy
percentile = lambda x: numpy.percentile(x,[50, 66, 90,95,99])
print(percentile(keyword_map[keyword_map['agriculture'] > 0]['agriculture'].values.tolist()))
print(percentile(keyword_map[keyword_map['difference'] > 0]['agriculture'].values.tolist()))
print(percentile(keyword_map[keyword_map['difference'] < 0]['agriculture'].values.tolist()))

numpy.mean(keyword_map[keyword_map['agriculture'] > 0]['agriculture'].values.tolist())

[ 0.43165468  0.86330935  2.87769784  4.75539568 12.23165468]
[ 0.43165468  0.71942446  2.58992806  4.60431655 11.65467626]
[0.         0.43165468 1.8705036  3.43165468 9.0647482 ]


1.2771223021582736

# Entrainement

In [9]:
def get_topic(e):
    if e >0:
        return 'agriculture'
    if e<0:
        return 'media'
    
def true_positve(a,b):
    if a == b:
        return 1
    else:
        return 0
    
    
n_components = 2
n_top_words = 20
n_features = 20000
text_corpus = df[df['categorie'].isin(['agriculture','media'])]['collection_counter']
model = model_lib.Models(n_components = n_components, n_top_words =n_top_words, n_features = n_features)
models = model.run_model_NMF(text_corpus.values.tolist())

done in 0.000s.
Extracting tf-idf features for NMF...
done in 0.146s.
Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=20000...
done in 0.217s.
Topic #0: product agricultural production market produce food farmer farm make create agriculture local service organic rural sustainable vegetable develop order young
Topic #1: communication digital agency graphic advertising event service design web information video platform tv public strategy marketing magazine social report documentary



In [10]:
predict = pandas.DataFrame(model.reverse_nmf(text_corpus.values.tolist(), models) , index =text_corpus.index)
predict

,0,1
key_main,,
http://chapwifi.bj/_adjassa_azimath,0.027398,0.117912
https://www.facebook.com/TZenSn/_diop_sokhnalaye,0.040763,0.000000
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,0.093996,0.033834
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,0.061235,0.000000
http://www.retpaci.ci_kouassi_ayagazalo,0.075245,0.000000
...,...,...
http://www.cbdibaong.org_houedanou_martin,0.064293,0.000000
http://www.cbdibaong.org_yemadje_emeline,0.038915,0.000000
http://www.cbdibaong.org_akokponsode_hippolyte,0.056188,0.000000


In [11]:
predict = predict.join(df['categorie'])
predict[0] = predict[0]*100
predict[1] = predict[1]*100
predict['topic'] = predict[0]-predict[1]
predict

,0,1,categorie,topic
key_main,,,,
http://chapwifi.bj/_adjassa_azimath,2.739772,11.791162,media,-9.051390
https://www.facebook.com/TZenSn/_diop_sokhnalaye,4.076287,0.000000,agriculture,4.076287
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,9.399576,3.383418,agriculture,6.016158
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,6.123518,0.000000,agriculture,6.123518
http://www.retpaci.ci_kouassi_ayagazalo,7.524515,0.000000,agriculture,7.524515
...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,6.429317,0.000000,agriculture,6.429317
http://www.cbdibaong.org_yemadje_emeline,3.891472,0.000000,agriculture,3.891472
http://www.cbdibaong.org_akokponsode_hippolyte,5.618756,0.000000,agriculture,5.618756


In [12]:
predict['predict_label'] = predict['topic'].map(get_topic)
predict

,0,1,categorie,topic,predict_label
key_main,,,,,
http://chapwifi.bj/_adjassa_azimath,2.739772,11.791162,media,-9.051390,media
https://www.facebook.com/TZenSn/_diop_sokhnalaye,4.076287,0.000000,agriculture,4.076287,agriculture
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,9.399576,3.383418,agriculture,6.016158,agriculture
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,6.123518,0.000000,agriculture,6.123518,agriculture
http://www.retpaci.ci_kouassi_ayagazalo,7.524515,0.000000,agriculture,7.524515,agriculture
...,...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,6.429317,0.000000,agriculture,6.429317,agriculture
http://www.cbdibaong.org_yemadje_emeline,3.891472,0.000000,agriculture,3.891472,agriculture
http://www.cbdibaong.org_akokponsode_hippolyte,5.618756,0.000000,agriculture,5.618756,agriculture


In [13]:
predict['classification_kpi'] = predict.apply(lambda x: true_positve(x['categorie'],x['predict_label']), axis= 1)
predict

,0,1,categorie,topic,predict_label,classification_kpi
key_main,,,,,,
http://chapwifi.bj/_adjassa_azimath,2.739772,11.791162,media,-9.051390,media,1
https://www.facebook.com/TZenSn/_diop_sokhnalaye,4.076287,0.000000,agriculture,4.076287,agriculture,1
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,9.399576,3.383418,agriculture,6.016158,agriculture,1
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,6.123518,0.000000,agriculture,6.123518,agriculture,1
http://www.retpaci.ci_kouassi_ayagazalo,7.524515,0.000000,agriculture,7.524515,agriculture,1
...,...,...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,6.429317,0.000000,agriculture,6.429317,agriculture,1
http://www.cbdibaong.org_yemadje_emeline,3.891472,0.000000,agriculture,3.891472,agriculture,1
http://www.cbdibaong.org_akokponsode_hippolyte,5.618756,0.000000,agriculture,5.618756,agriculture,1


In [15]:
agri = predict[predict['categorie'] =='agriculture']
print('precision {}'.format(agri['classification_kpi'].sum()/695*100))

errors = agri[agri['classification_kpi'] == 0].sort_values('topic')

collections.Counter(' '.join(errors.join(text_corpus)['collection_counter'].values.tolist()).split()).most_common(10)

precision 97.98561151079137


[('communication', 7),
 ('product', 5),
 ('agricultural', 5),
 ('service', 5),
 ('market', 4),
 ('digital', 4),
 ('platform', 4),
 ('local', 4),
 ('information', 4),
 ('create', 4)]

In [16]:
template = errors.join(chalenge1000.Native().text['temp'])['temp'].values.tolist()
with open('misclassified_agri.txt', 'w') as file:
    for line in template:
        file.writelines(line)

# catégorie abscente

In [27]:
label = pandas.read_csv('datastore.csv').set_index('key_main')
label['corpus'] = label[label['_agriculture_']][['prez_struc','prez_produit_struc']].agg(sum, axis = 1)
label['type'] = label['corpus'].map(type)
label['corpus'] = label['corpus'].map(vocab)
label['corpus_lemm'] = label['corpus'].map(lemm)
label['collection_counter'] = label['corpus_lemm'].map(col_count)
label = label[label['type'] ==str]

n_components = 2
n_top_words = 20
n_features = 20000
text_corpus = label['collection_counter']

predict = pandas.DataFrame(model.reverse_nmf(text_corpus.values.tolist(), models) , index =text_corpus.index)
predict = predict.join(df['categorie'])
predict[0] = predict[0]*100
predict[1] = predict[1]*100
predict['topic'] = predict[0]-predict[1]
predict['predict_label'] = predict['topic'].map(get_topic)
predict['classification_kpi'] = predict.apply(lambda x: true_positve(x['categorie'],x['predict_label']), axis= 1)
predict.sort_values(0, ascending = False)[[0,1,'predict_label','topic']]

,0,1,predict_label,topic
key_main,,,,
http://www.aquareignindustry.com_mbouleediango_marcidriss,6.607159,0.000000,agriculture,6.607159
https://web.facebook.com/faycal.aguemon_aguemon_shegun,5.565335,0.000000,agriculture,5.565335
https://www.2mi-sa.com/_djermakoye_reki,3.547847,0.000000,agriculture,3.547847
http://www.congofresh.com_enge_albert,3.477652,0.000000,agriculture,3.477652
https://www.youtube.com/results?search_query=gue+douade+marius+biographie_gue_douademarius,2.957206,0.000000,agriculture,2.957206
http://www.leader-manioc-96.websel.net/site%20map.xml_adognon_alidaodilekayissan,2.863441,0.000000,agriculture,2.863441
https://4piment.blogspot.com/_sinare_idrissa,2.133455,0.000000,agriculture,2.133455
http://YamAgroIndustrie.bf_yameogo_stevearmandleonardwendkouni,1.945061,0.000000,agriculture,1.945061
https://www.le-lionceau.com/_samba_siny,1.861991,6.776858,media,-4.914867


In [26]:
for e in predict[predict['predict_label'] == 'agriculture'].join(Chalenge.descriptions_trad)[['prez_struc','prez_produit_struc']].values.tolist()[0]:
    print(e)

Company specialized in the food processing made in Burkina Faso, we offer to the Burkinabe consumers syrup, biscuits and jam.
We have two types of syrup, bissap and ginger. One is the chocolate cookies and one is mango jam. We plan to launch three new syrups and one jam on the market by March 2020.


# Startup non inscrits en agriculture

In [26]:
df = Chalenge.X
df['corpus'] = df.join(Chalenge.descriptions_trad)[['prez_struc','prez_produit_struc']].agg(sum, axis = 1)
df['type'] = df['corpus'].map(type)
df['corpus'] = df['corpus'].map(vocab)
df['corpus_lemm'] = df['corpus'].map(lemm)
df['collection_counter'] = df['corpus_lemm'].map(col_count)
df = df[df['type'] ==str]

n_components = 2
n_top_words = 20
n_features = 20000
text_corpus = df[df['categorie']!= 'agriculture']['collection_counter']

predict = pandas.DataFrame(model.reverse_nmf(text_corpus.values.tolist(), models) , index =text_corpus.index)
predict = predict.join(df['categorie'])
predict[0] = predict[0]*100
predict[1] = predict[1]*100
predict['topic'] = predict[0]-predict[1]
predict['predict_label'] = predict['topic'].map(get_topic)
predict['classification_kpi'] = predict.apply(lambda x: true_positve(x['categorie'],x['predict_label']), axis= 1)
#predict.sort_values(0, ascending = False)[[0,1,'predict_label']]

In [27]:
#predict.sort_values(0, ascending = False).describe()

In [28]:
#predict[predict['topic'] > 10].sort_values(0, ascending = False).groupby(['categorie','predict_label']).nunique()#.describe(include ='all')

In [29]:
predict[predict['topic']>10].join(chalenge1000.Native().text['temp'])[['categorie','predict_label']]

,categorie,predict_label
key_main,,
http://www.sukuchai.com_kuchengo_suleiman,autres,agriculture
https://www.facebook.com/Community-Action-for-Agriculture-and-Sustainable-Development-466865864068319/_noellifogha_ndze,education,agriculture
http://www.cleanprogb.com%20en%20contruction_etimbibang_confort,finance,agriculture
https://afrikapu.com/_mohamed_nasreenali,ecommerce,agriculture
https://www.facebook.com/TESSA-POWER-SARL-1540172142980252/?ref=bookmarks_ousmane_mahamanlaouali,energie,agriculture
...,...,...
https://solarkoodo.com/_nana_safiatou,energie,agriculture
http://www.agrolimgroceriesuganda.com/_denis_muhereza,ecommerce,agriculture
http://www.afrimart.com.ng/_ojeniyi_olajide,ecommerce,agriculture


In [31]:
a = predict[predict['topic']>10].join(chalenge1000.Native().text['temp'])['temp'].values.tolist()
print(a[0])

Nom du répondant : Kuchengo 
Prénom du répondant : Suleiman 
email : suleykuchengo@yahoo.com 
 
Nom de la structure : SUKU CHAI 
Pays d'origine : Tanzania 
Pays d'activité : Tanzania 
Site internet : http://www.sukuchai.com 
Linkedin : https://www.linkedin.com/in/suleiman-kuchengo-05a370123/?originalSubdomain=tz 
 
Présentation de la structure 
Our main goal is to improve lives in remote, impoverished and neglected rural areas by creating improved access to productive resources as well as meaningful jobs to single mothers, widows, people with disabilities and young girls through tea farming and value addition.
Our approach is based on building an inclusive economy as we are challenging and disrupting the status quo by changing the way marginalized groups and excluded communities were left in the stake of the economy by giving them the necessary skills, tools, and resources to own the economy.
What we do is to empower those marginalized and neglected groups through training and incentiv